In [ ]:
import os
from transformers import AutoTokenizer
from datasets import Dataset, Features, Value, Sequence
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
datapath = '/content/drive/MyDrive/datasets/SQuAD-v1.zip'
extract_path = '/content/dataset'
with zipfile.ZipFile(datapath, 'r') as zipref:
  zipref.extractall(extract_path)


In [ ]:
datapath = '/content/dataset/'
os.listdir(datapath)

['train-v1.1.json', 'dev-v1.1.json']

In [ ]:
train_set = datapath + 'train-v1.1.json'
with open(train_set, 'r') as f:
  train_data = json.load(f)

In [ ]:
for i in train_data:
  print(i)

data
version


In [ ]:
print(len(train_data['data']))
print(len(train_data['version']))

442
3


In [ ]:
for i in train_data['data']:
  print(i)
  break

{'title': 'University_of_Notre_Dame', 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'qas': [{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 

In [ ]:
for i in train_data['data']:
  sample_dict = json.dumps(i,indent=4)
  print(sample_dict)
  break

{
    "title": "University_of_Notre_Dame",
    "paragraphs": [
        {
            "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
            "qas": [
                {
                    "answers": [
                        {
                            "answer_start": 515,
                            "text": "Saint Bernadette So

In [ ]:
# Checking the Answer span correctly

question = train_data['data'][0]['paragraphs'][0]['qas'][0]['question']
answer = train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
answer_start =  train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
context = train_data['data'][0]['paragraphs'][0]['context']
print(context)
print(context[answer_start:])
print(question)
print(answer)

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Saint Bernadette Soubirous


For Understanding and easiness we will preprocessing the only above data (example question, context, answer)

In [ ]:
# 1. Initialize the Tokenizer
model_name = "bert-base-uncased" # Use a specific pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_example = tokenizer(
    question,
    context,
    truncation="only_second", # Truncate the context (second argument)
    max_length=512,
    stride=128,
    return_overflowing_tokens=True, # Important for long contexts
    return_offsets_mapping=True, # Crucial for alignment
    padding="max_length"
)

print("Tokenized Sequence (Input IDs):", tokenized_example['input_ids'][0])
print("Token Segment IDs:", tokenized_example['token_type_ids'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenized Sequence (Input IDs): [101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 113

In [ ]:
feature_index = 0
offsets = tokenized_example['offset_mapping'][feature_index]
sequence_ids = tokenized_example['token_type_ids'][feature_index]

# Calculate the character end index of the answer
answer_end = answer_start + len(answer)

# Initialize start/end token indices
start_token_index = 0
end_token_index = 0

context_start_token = sequence_ids.index(1)
context_end_token = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

for i in range(context_start_token, context_end_token + 1):
    token_start_char, token_end_char = offsets[i]

    if start_token_index == 0 and token_start_char <= answer_start < token_end_char:
        start_token_index = i

    if token_start_char < answer_end <= token_end_char:
        end_token_index = i
print(f"Answer Start Character Index: {answer_start}, End Character Index: {answer_end}")
print(f"Answer Start Token Index: {start_token_index}, End Token Index: {end_token_index}")

Answer Start Character Index: 515, End Character Index: 541
Answer Start Token Index: 130, End Token Index: 137


In [ ]:
input_ids = tokenized_example['input_ids'][feature_index]
token_type_ids = tokenized_example['token_type_ids'][feature_index]
attention_mask = tokenized_example['attention_mask'][feature_index]

start_positions = start_token_index
end_positions = end_token_index

processed_feature = {
    'input_ids': input_ids,
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask,
    'start_positions': start_positions,
    'end_positions': end_positions,
    'example_id': 'unique_squad_id_001' # Keep original ID for mapping predictions back
}

print("\n--- Example Feature for Training ---")
print(f"Input IDs Length: {len(processed_feature['input_ids'])}")
print(f"Start Position: {processed_feature['start_positions']}")
print(f"End Position: {processed_feature['end_positions']}")


--- Example Feature for Training ---
Input IDs Length: 512
Start Position: 130
End Position: 137


In [ ]:

all_features = [processed_feature]

feature_schema = Features({
    'input_ids': Sequence(Value('int32')),
    'token_type_ids': Sequence(Value('int8')),
    'attention_mask': Sequence(Value('int8')),
    'start_positions': Value('int32'),
    'end_positions': Value('int32'),
    'example_id': Value('string')
})

# Creating the Dataset
dataset = Dataset.from_list(all_features, features=feature_schema)

print("\n--- Final Dataset Object ---")
print(dataset)
print("First Feature in Dataset:")
print(dataset[0])


--- Final Dataset Object ---
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'example_id'],
    num_rows: 1
})
First Feature in Dataset:
{'input_ids': [101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 259